<a href="https://colab.research.google.com/github/philptt/CustomEfficientDet/blob/main/SIIM_EffDet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data preparation

In [ ]:
## packages

import numpy as np
import pandas as pd
from ast import literal_eval
import shutil
import random
import hashlib

# File import
import os
import io
import glob


#model
import tensorflow as tf

#images
import PIL.Image

In [ ]:
# Unzip data and prepare folders

if not os.path.exists("images"):
    os.mkdir("images")

if not os.path.exists("labels"):
    os.mkdir("labels")

!unzip /content/drive/MyDrive/resized.zip -d "images"

In [ ]:
# Image training data

df_image = pd.read_csv("/content/drive/MyDrive/siim-covid19-detection/train_image_level.csv")
df_image["id"] = df_image["id"].str.replace("_image","")

# Study training data

df_study = pd.read_csv("/content/drive/MyDrive/siim-covid19-detection/train_study_level.csv")
df_study["id"] = df_study["id"].str.replace("_study","")
df_study.rename({"id": "StudyInstanceUID"}, axis=1, inplace=True)

# Merge data

df_merge = df_image.merge(df_study, on="StudyInstanceUID")


# Original sizes (before resizing to 256*256)

sizes = pd.read_csv("images/sizes.csv")

# Merge data and original sizes

df_with_sizes = df_merge.merge(sizes, on="id")
df_with_sizes.head(2)

,id,boxes,label,StudyInstanceUID,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance,width0,height0
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,0,1,0,0,4256,3488
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,1,0,0,0,2832,2320


In [ ]:
# Function to convert boxes to xmin xmax ymin ymax format

def convert_box(box):
    xmin = box["x"]
    ymin = box["y"]
    w = box["width"]
    h = box["height"]
    xmax = (box["x"] + w)
    ymax = (box["y"] + h)
    coord_dict = {"xmin":xmin, "ymin":ymin, "xmax":xmax, "ymax":ymax}
    return coord_dict

# Creating list of dicts with id and lists of box coordinates xmins, ymins, xmaxs, ymaxs
# This format is used in TFRecords for multi-object detection

list_id_coord = []
for i in range(df_with_sizes.shape[0]):
    if type(df_with_sizes["boxes"][i]) == str:
        boxes = literal_eval(df_with_sizes["boxes"][i])
        name = df_with_sizes.iloc[i]["id"]
        box_list = []
        xmins=[]
        xmaxs=[]
        ymins=[]
        ymaxs=[]
        for i, box in enumerate(boxes):
            coords = convert_box(box)
            box_list.append(coords)
            xmins.append(box_list[i]["xmin"])
            ymins.append(box_list[i]["ymin"])
            xmaxs.append(box_list[i]["xmax"])
            ymaxs.append(box_list[i]["ymax"])

        dict_coord = {"id":name, "xmins0":xmins, "ymins0":ymins, "xmaxs0":xmaxs, "ymaxs0":ymaxs}
        list_id_coord.append(dict_coord)
    else:
        name = df_with_sizes.iloc[i]["id"]
        dict_coord = {"id":name, "xmins0":np.NaN, "ymins0":np.NaN, "xmaxs0":np.NaN, "ymaxs0":np.NaN}
        list_id_coord.append(dict_coord)


In [ ]:
# Merging list_id_coords with df_train file to gather data alltogether

df_box_coords = pd.DataFrame(list_id_coord)
df_total = df_with_sizes.merge(df_box_coords, on="id")

# Selecting images with bounding boxes
df_withboxes = df_total[df_total["boxes"].apply(lambda x: isinstance(x, str))]

print("df_total:",df_total.shape)
print("df_withboxes:",df_withboxes.shape)


df_total: (6334, 14)
df_withboxes: (4294, 14)


In [ ]:
# Train/Val split and save to Google Drive

img_dir = "/content/drive/MyDrive/siim-covid19-detection/images/" 
label_dir = "/content/drive/MyDrive/siim-covid19-detection/labels/"


src = "images/*.png"
allFileNames = glob.glob(src)

# Creating partitions of the data after shuffeling
val_ratio = 0.15
df_withboxes_shuffle = df_withboxes.sample(frac=1, random_state=42)
df_train, df_val = np.split(df_withboxes_shuffle, [int(df_withboxes_shuffle.shape[0] * (1 - val_ratio))])
print("Shape of df_withboxes_shuffle: ", df_withboxes_shuffle.shape)
print("Shape of df_train: ", df_train.shape)
print("Shape of df_val: ", df_val.shape)

df_train.to_csv("/content/drive/MyDrive/siim-covid19-detection/labels/train/df_train.csv")
df_val.to_csv("/content/drive/MyDrive/siim-covid19-detection/labels/val/df_val.csv")



Shape of df_withboxes_shuffle:  (4294, 14)
Shape of df_train:  (3649, 14)
Shape of df_val:  (645, 14)


In [ ]:
# Copy-pasting images
for name in df_train["id"]:
    shutil.copy(str("images/" + name + ".png"), img_dir + "train/")

for name in df_val["id"]:
    shutil.copy(str("images/" + name + ".png"), img_dir + "val/")

In [ ]:
# file checkings

train_files = "/content/drive/MyDrive/siim-covid19-detection/images/train/*.png"
val_files = "/content/drive/MyDrive/siim-covid19-detection/images/val/*.png"
train_filepaths = glob.glob(train_files)
val_filepaths = glob.glob(val_files)

print("number of train images", len(train_filepaths))
print("number of val images", len(val_filepaths))

number of train images 3649
number of val images 645


In [ ]:
# Creating TFRecords 
# Official TF Doc (does not work !) for multi-object detection here:
# https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/using_your_own_dataset.md

def create_tf_example(filepath, df_label):

    encoded_image_data = open(filepath, "rb").read()
    key = hashlib.sha256(encoded_image_data).hexdigest()
    filename = os.path.basename(filepath)
    image_name = filename.replace(".png", "")
    height0 = df_label["height0"].loc[df_label["id"]==image_name].iloc[0]
    width0 = df_label["width0"].loc[df_label["id"]==image_name].iloc[0]
    image_format = b'png'
    width = 256
    height = 256

    xmins = [x / width0 for x in df_label["xmins0"].loc[df_label["id"]==image_name].iloc[0]]
    xmaxs = [x / width0 for x in df_label["xmaxs0"].loc[df_label["id"]==image_name].iloc[0]]
    ymins = [x / height0 for x in df_label["ymins0"].loc[df_label["id"]==image_name].iloc[0]]
    ymaxs = [x / height0 for x in df_label["ymaxs0"].loc[df_label["id"]==image_name].iloc[0]]
    classes_text = ["opacity".encode("utf-8")]
    classes = [1]

    tf_example = tf.train.Example(features=tf.train.Features(feature={
        'image/height': tf.train.Feature(int64_list=tf.train.Int64List(value=[height])),
        'image/width': tf.train.Feature(int64_list=tf.train.Int64List(value=[width])),
        "image/filename": tf.train.Feature(bytes_list=tf.train.BytesList(value=[filename.encode("utf-8")])),
        "image/source_id": tf.train.Feature(bytes_list=tf.train.BytesList(value=['0'.encode("utf-8")])), # Pb with image names solved with this hack
        "image/key/sha256": tf.train.Feature(bytes_list=tf.train.BytesList(value=[key.encode("utf-8")])),
        "image/encoded": tf.train.Feature(bytes_list=tf.train.BytesList(value=[encoded_image_data])),
        "image/format": tf.train.Feature(bytes_list=tf.train.BytesList(value=["png".encode("utf-8")])),
        "image/object/bbox/xmin": tf.train.Feature(float_list=tf.train.FloatList(value=xmins)),
        "image/object/bbox/xmax": tf.train.Feature(float_list=tf.train.FloatList(value=xmaxs)),
        "image/object/bbox/ymin": tf.train.Feature(float_list=tf.train.FloatList(value=ymins)),
        "image/object/bbox/ymax": tf.train.Feature(float_list=tf.train.FloatList(value=ymaxs)),
        "image/object/class/text": tf.train.Feature(bytes_list=tf.train.BytesList(value=classes_text)),
        "image/object/class/label": tf.train.Feature(int64_list=tf.train.Int64List(value=classes)),
        }))
    return tf_example




In [ ]:
# Writing train dataset

writer_train = tf.io.TFRecordWriter('/content/drive/MyDrive/siim-covid19-detection/TFRecords/train/train.tfrecord')

for filepath in train_filepaths:
    tf_example = create_tf_example(filepath, df_train)
    writer_train.write(tf_example.SerializeToString())

writer_train.close()

In [ ]:
# Same for val dataset

writer_val = tf.io.TFRecordWriter('/content/drive/MyDrive/siim-covid19-detection/TFRecords/val/val.tfrecord')
for filepath in val_filepaths:
    tf_example = create_tf_example(filepath, df_val)
    writer_val.write(tf_example.SerializeToString())

writer_val.close()

In [ ]:
#If you wish to inspect the tfrecord file

raw_dataset = tf.data.TFRecordDataset("/content/drive/MyDrive/siim-covid19-detection/TFRecords/val/val.tfrecord")

for raw_record in raw_dataset.take(1):
    example = tf.train.Example()
    example.ParseFromString(raw_record.numpy())
    print(example)

features {
  feature {
    key: "image/encoded"
    value {
      bytes_list {
        value: "\211PNG\r\n\032\n\000\000\000\rIHDR\000\000\001\000\000\000\001\000\010\000\000\000\000y\031\367\272\000\000 \000IDATx\001\274\301\311\216\256k\226\030\344\265\336\366k\3776\272\335\235&\363Tf\222.[B\226\021\202\021\027\300\204)w\200\204\270\003B\201\020\003\317<g\200\000\001\202\221%\013\311#d\203d,P\271ZW\223\'O\263\233\330;\"\376\366k\337v-vfeV\271\212*\224%\362\324\363 \374\332!\010B\000@dD\001\005Y\261\216\251\004im\330\204\375\225|\255\'+\014\364Y2-\233p\371nV=\323\000Y\304\314@\300\000\014\177[\020~\335\020\220\021\000\001\031Q\260\222\254uk\211[}6\004\355\271\214~\371\250\244\220\303`\270\250\200 \250,\036!\311\230!\0230\301G\014\177K\020~\275\020\000\001\020\220\021\001Y\263\220 \033Y\214\313d\303\246\2576)\034[\261GW\245~\221Y\274<G\021\314\340f=\023\004\316\014\304\300\000\300\360\267\003\341\327\006\201\021\020\020Ae\020\250\000I\223\304\206T{\211\222\364\302\207\272\223A\230\235_

### EfficientDet

In [ ]:
# Try this bug fix to solve error
# Invalid argument: indices[2] = [2] does not index into param shape [1,1], node name: parser/GatherNd_1
#	 [[{{node parser/GatherNd_1}}]]

gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)

RuntimeError: ignored

In [ ]:
# Model download

if not os.path.isdir("automl"):
    # clone the repository
    !git clone --depth 1 https://github.com/google/automl
        
    # checkout to the latest commit that we used when we're creating this notebook
    %cd automl
    !git checkout f2b4480703278250fb05abe38a2f4ecbb16ba463
    
    # change the working directory
    %cd efficientdet
    
    # install required packages
    # if it fails to install pycocotools, please manually remove pycocotools from requirements.txt and run again
    %pip install -r requirements.txt
    %pip install -U "git+https://github.com/cocodataset/cocoapi.git#subdirectory=PythonAPI"



Cloning into 'automl'...
remote: Enumerating objects: 153, done.
remote: Counting objects: 100% (153/153), done.
remote: Compressing objects: 100% (149/149), done.
remote: Total 153 (delta 17), reused 43 (delta 0), pack-reused 0
Receiving objects: 100% (153/153), 11.47 MiB | 25.04 MiB/s, done.
Resolving deltas: 100% (17/17), done.
/content/automl
fatal: reference is not a tree: f2b4480703278250fb05abe38a2f4ecbb16ba463
/content/automl/efficientdet
  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-vk7ettq_
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-vk7ettq_
     |████████████████████████████████| 6.3MB 7.4MB/s 
     |████████████████████████████████| 645kB 13.5MB/s 
     |████████████████████████████████| 686kB 13.6MB/s 
     |████████████████████████████████| 122kB 13.6MB/s 
     |████████████████████████████████| 174kB 13.4MB/s 
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.wh

  Cloning https://github.com/cocodataset/cocoapi.git to /tmp/pip-req-build-yh2frp87
  Running command git clone -q https://github.com/cocodataset/cocoapi.git /tmp/pip-req-build-yh2frp87
  Created wheel for pycocotools: filename=pycocotools-2.0-cp37-cp37m-linux_x86_64.whl size=263902 sha256=0203eb7c30a73b7831e2fa7df7bade8310764d7b0ba6b46eb117eae0c21634e0
  Stored in directory: /tmp/pip-ephem-wheel-cache-fw3amwxi/wheels/90/51/41/646daf401c3bc408ff10de34ec76587a9b3ebfac8d21ca5c3a
Successfully built pycocotools
  Found existing installation: pycocotools 2.0
    Uninstalling pycocotools-2.0:
      Successfully uninstalled pycocotools-2.0


In [ ]:
# Download pre-trained model
# Chosen model : EfficientDet-d0

MODEL = "efficientdet-d0"
if not os.path.exists(f"{MODEL}.tar.gz"):
    !curl -O https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco2/{MODEL}.tar.gz
    !tar xvzf {MODEL}.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 27.4M  100 27.4M    0     0  50.7M      0 --:--:-- --:--:-- --:--:-- 50.6M
efficientdet-d0/
efficientdet-d0/model.meta
efficientdet-d0/model.index
efficientdet-d0/checkpoint
efficientdet-d0/d0_coco_val.txt
efficientdet-d0/model.data-00000-of-00001
efficientdet-d0/d0_coco_test-dev2017.txt


In [ ]:
# !python model_inspect.py --model_name={MODEL} --logdir=logs &> /dev/null
# %load_ext tensorboard
# %tensorboard --logdir logs

In [ ]:
# Create YAML file

PROJ_DIR = "/content/MODEL"
CONFIG_DIR = os.path.join(PROJ_DIR, "configs")
CONFIG_FILE = os.path.join(CONFIG_DIR, "default.yaml")
if not os.path.exists(CONFIG_DIR):
    os.mkdir(CONFIG_DIR)

config_text = \
"""image_size: 256x256
num_classes: 1
label_map: {1: opacity}
input_rand_hflip: true
jitter_min: 0.8
jitter_max: 1.2
"""

with open(CONFIG_FILE, "w") as fwrite:
    fwrite.write(config_text)

In [ ]:
CONFIG_FILE

'/content/MODEL/configs/default.yaml'

In [ ]:
# Training parameters

TFRECORD_DIR = "/content/drive/MyDrive/siim-covid19-detection/TFRecords"

CKPT = MODEL

TRAIN_SET = os.path.join(TFRECORD_DIR, "train/train.tfrecord")
VAL_SET = os.path.join(TFRECORD_DIR, "val/val.tfrecord")
MODEL_DIR_TMP = os.path.join(PROJ_DIR, "tmp", f"{MODEL}-finetune")
TRAIN_NUM_EXAMPLES = len(train_filepaths)
EVAL_NUM_EXAMPLES = len(val_filepaths)
EPOCHS = 2
BATCH_SIZE = 16

In [ ]:
VAL_SET

'/content/drive/MyDrive/siim-covid19-detection/TFRecords/val/val.tfrecord'

In [ ]:
# Remove previous checkpoints

if os.path.exists(MODEL_DIR_TMP):
    !rm -rf {MODEL_DIR_TMP}

In [ ]:
# MODEL TRAINING

!python -m main \
    --mode=train_and_eval \
    --train_file_pattern={TRAIN_SET} \
    --val_file_pattern={VAL_SET} \
    --model_name={MODEL} \
    --model_dir={MODEL_DIR_TMP} \
    --ckpt={CKPT} \
    --train_batch_size={BATCH_SIZE} \
    --eval_batch_size={BATCH_SIZE} \
    --num_epochs={EPOCHS} \
    --num_examples_per_epoch={TRAIN_NUM_EXAMPLES} \
    --eval_samples={EVAL_NUM_EXAMPLES} \
    --hparams={CONFIG_FILE}

2021-06-17 20:22:05.257843: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
I0617 20:22:06.782572 140287598352256 main.py:228] {'name': 'efficientdet-d0', 'act_type': 'swish', 'image_size': (256, 256), 'target_size': None, 'input_rand_hflip': True, 'jitter_min': 0.8, 'jitter_max': 1.2, 'autoaugment_policy': None, 'grid_mask': False, 'sample_image': None, 'map_freq': 5, 'num_classes': 1, 'seg_num_classes': 3, 'heads': ['object_detection'], 'skip_crowd_during_training': True, 'label_map': {1: 'opacity'}, 'max_instances_per_image': 100, 'regenerate_source_id': False, 'min_level': 3, 'max_level': 7, 'num_scales': 3, 'aspect_ratios': [1.0, 2.0, 0.5], 'anchor_scale': 4.0, 'is_training_bn': True, 'momentum': 0.9, 'optimizer': 'sgd', 'learning_rate': 0.08, 'lr_warmup_init': 0.008, 'lr_warmup_epoch': 1.0, 'first_lr_drop_epoch': 200.0, 'second_lr_drop_epoch': 250.0, 'poly_lr_power': 0.9, 'clip_gradients_norm': 10.0, 'num_epoch

In [ ]:
python dataset/inspect_tfrecords.py --file_pattern dataset/sample.record\ 
--model_name "efficientdet-d0" --samples 10\ 
--save_samples_dir train_samples/  -hparams="label_map={1:'label1'}, autoaugmentation_policy=v3"

SyntaxError: ignored